In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-2.4.8'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-23 07:17:50--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.92MB/s    in 0.2s    

2022-01-23 07:17:51 (5.92 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-23 07:17:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  6.05MB/s    in 0.2s    

2022-01-23 07:17:51 (6.05 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
import pandas as pd
from typing import List

urlList = ["https://group8-bucket.s3.amazonaws.com/SPX_Health_Sector_stocks_all.csv",
"https://group8-bucket.s3.amazonaws.com/SPX_Health_Sector_stocks.csv",
"https://group8-bucket.s3.amazonaws.com/Covid_Companies_Stocks.csv",
"https://group8-bucket.s3.amazonaws.com/SPX_Health_Sector_info.csv",
"https://group8-bucket.s3.amazonaws.com/sp500indexand11sectors/S5COND_INDEX.csv",
"https://group8-bucket.s3.amazonaws.com/sp500indexand11sectors/S5CONS_INDEX.csv",
"https://group8-bucket.s3.amazonaws.com/sp500indexand11sectors/S5ENRS_INDEX.csv",
"https://group8-bucket.s3.amazonaws.com/sp500indexand11sectors/S5FINL_INDEX.csv",
"https://group8-bucket.s3.amazonaws.com/sp500indexand11sectors/S5HLTH_INDEX.csv",
"https://group8-bucket.s3.amazonaws.com/sp500indexand11sectors/S5INDU_INDEX.csv",
"https://group8-bucket.s3.amazonaws.com/sp500indexand11sectors/S5INFT_INDEX.csv",
"https://group8-bucket.s3.amazonaws.com/sp500indexand11sectors/S5MATR_INDEX.csv",
"https://group8-bucket.s3.amazonaws.com/sp500indexand11sectors/S5RLST_Index.csv",
"https://group8-bucket.s3.amazonaws.com/sp500indexand11sectors/S5TELS_INDEX.csv",
"https://group8-bucket.s3.amazonaws.com/sp500indexand11sectors/S5UTIL_INDEX.csv",
"https://group8-bucket.s3.amazonaws.com/sp500indexand11sectors/SPX_INDEX.csv" ]

for x in urlList:
   spark.sparkContext.addFile(x)

#ToDo convert to loop
user_data_df =  spark.read.csv(SparkFiles.get("SPX_Health_Sector_stocks_all.csv"), sep=",", header=True, inferSchema=True)
user_data1_df = spark.read.csv(SparkFiles.get("SPX_Health_Sector_stocks.csv"), sep=",", header=True, inferSchema=True)
user_data2_df = spark.read.csv(SparkFiles.get("Covid_Companies_Stocks.csv"), sep=",", header=True, inferSchema=True)
user_data3_df = spark.read.csv(SparkFiles.get("SPX_Health_Sector_info.csv"), sep=",", header=True, inferSchema=True)
user_data4_df = spark.read.csv(SparkFiles.get("S5COND_INDEX.csv"), sep=",", header=True, inferSchema=True)
user_data5_df = spark.read.csv(SparkFiles.get("S5CONS_INDEX.csv"), sep=",", header=True, inferSchema=True)
user_data6_df = spark.read.csv(SparkFiles.get("S5ENRS_INDEX.csv"), sep=",", header=True, inferSchema=True)
user_data7_df = spark.read.csv(SparkFiles.get("S5FINL_INDEX.csv"), sep=",", header=True, inferSchema=True)
user_data8_df = spark.read.csv(SparkFiles.get("S5HLTH_INDEX.csv"), sep=",", header=True, inferSchema=True)
user_data9_df = spark.read.csv(SparkFiles.get("S5INDU_INDEX.csv"), sep=",", header=True, inferSchema=True)
user_data10_df = spark.read.csv(SparkFiles.get("S5INFT_INDEX.csv"), sep=",", header=True, inferSchema=True)
user_data11_df = spark.read.csv(SparkFiles.get("S5MATR_INDEX.csv"), sep=",", header=True, inferSchema=True)
user_data12_df = spark.read.csv(SparkFiles.get("S5RLST_Index.csv"), sep=",", header=True, inferSchema=True)
user_data13_df = spark.read.csv(SparkFiles.get("S5TELS_INDEX.csv"), sep=",", header=True, inferSchema=True)
user_data14_df = spark.read.csv(SparkFiles.get("S5UTIL_INDEX.csv"), sep=",", header=True, inferSchema=True)
user_data15_df = spark.read.csv(SparkFiles.get("SPX_INDEX.csv"), sep=",", header=True, inferSchema=True)

In [7]:
#user_data_df['Date'] = pd.to_datetime(user_data_df['Date'], format="%y/%m/%d")
from pyspark.sql.functions import to_date

#ToDo: convert to loop
user_data_df = user_data_df.select(['Ticker', to_date('Date', 'YYY-MM-dd').alias("date"),'Open', 'High', 'Low', 'Close','Trading_Volume', 'Volume_Weighted_Average_Price' ,'Number_of_Transactions'])
user_data1_df = user_data1_df.select(['Ticker', to_date('Date', 'YYY-MM-dd').alias("date"),'Open', 'High', 'Low', 'Close','Trading_Volume', 'Volume_Weighted_Average_Price' ,'Number_of_Transactions'])
user_data2_df = user_data2_df.select(['Ticker', to_date('Date', 'YYY-MM-dd').alias("date"),'Open', 'High', 'Low', 'Close','Trading_Volume', 'Volume_Weighted_Average_Price' ,'Number_of_Transactions'])
user_data3_df = user_data3_df.select(['Ticker','Company', 'Sector'])
user_data4_df = user_data4_df.select([to_date('Date', 'YYYY-MM-dd').alias("date"), 'Open' , 'High', 'Low', 'Close'])
user_data5_df = user_data5_df.select([to_date('Date', 'YYYY-MM-dd').alias("date"), 'Open' , 'High', 'Low', 'Close'])
user_data6_df = user_data6_df.select([to_date('Date', 'YYYY-MM-dd').alias("date"), 'Open' , 'High', 'Low', 'Close'])
user_data7_df = user_data7_df.select([to_date('Date', 'YYYY-MM-dd').alias("date"), 'Open' , 'High', 'Low', 'Close'])
user_data8_df = user_data8_df.select([to_date('Date', 'YYYY-MM-dd').alias("date"), 'Open' , 'High', 'Low', 'Close'])
user_data9_df = user_data9_df.select([to_date('Date', 'YYYY-MM-dd').alias("date"), 'Open' , 'High', 'Low', 'Close'])
user_data10_df = user_data10_df.select([to_date('Date', 'YYYY-MM-dd').alias("date"), 'Open' , 'High', 'Low', 'Close'])
user_data11_df = user_data11_df.select([to_date('Date', 'YYYY-MM-dd').alias("date"), 'Open' , 'High', 'Low', 'Close'])
user_data12_df = user_data12_df.select([to_date('Date', 'YYYY-MM-dd').alias("date"), 'Open' , 'High', 'Low', 'Close'])
user_data13_df = user_data13_df.select([to_date('Date', 'YYYY-MM-dd').alias("date"), 'Open' , 'High', 'Low', 'Close'])
user_data14_df = user_data14_df.select([to_date('Date', 'YYYY-MM-dd').alias("date"), 'Open' , 'High', 'Low', 'Close'])
user_data15_df = user_data15_df.select([to_date('Date', 'YYYY-MM-dd').alias("date"), 'Open' , 'High', 'Low', 'Close'])

In [8]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cxmneo38riye.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [10]:
# Write user_data_df to table in RDS

#ToDo Convert to for loop
user_data_df.write.jdbc(url=jdbc_url, table='spx_health_sector_stocks_all', mode=mode, properties=config)
user_data1_df.write.jdbc(url=jdbc_url, table='spx_health_sector_stocks', mode=mode, properties=config)
user_data2_df.write.jdbc(url=jdbc_url, table='covid_companies_stocks', mode=mode, properties=config)
user_data3_df.write.jdbc(url=jdbc_url, table='spx_health_sector_Info', mode=mode, properties=config)
user_data4_df.write.jdbc(url=jdbc_url, table='s5cond_index', mode=mode, properties=config)
user_data5_df.write.jdbc(url=jdbc_url, table='s5cons_index', mode=mode, properties=config)
user_data6_df.write.jdbc(url=jdbc_url, table='s5enrs_index', mode=mode, properties=config)
user_data7_df.write.jdbc(url=jdbc_url, table='s5finl_index', mode=mode, properties=config)
user_data8_df.write.jdbc(url=jdbc_url, table='s5hlth_index', mode=mode, properties=config)
user_data9_df.write.jdbc(url=jdbc_url, table='s5indu_index', mode=mode, properties=config)
user_data10_df.write.jdbc(url=jdbc_url, table='s5inft_index', mode=mode, properties=config)
user_data11_df.write.jdbc(url=jdbc_url, table='s5matr_index', mode=mode, properties=config)
user_data12_df.write.jdbc(url=jdbc_url, table='s5rlst_index', mode=mode, properties=config)
user_data13_df.write.jdbc(url=jdbc_url, table='s5tels_index', mode=mode, properties=config)
user_data14_df.write.jdbc(url=jdbc_url, table='s5util_index', mode=mode, properties=config)
user_data15_df.write.jdbc(url=jdbc_url, table='spx_index', mode=mode, properties=config)
